# 👶 Fetal Health Classification Analysis
## PACE Yaklaşımı ile End-to-End Veri Bilimi Projesi

---

### 📋 Proje Genel Bakış

Bu proje, **PACE (Plan, Analyze, Construct, Execute)** metodolojisi kullanarak fetal health sınıflandırma modellemesi yapmayı amaçlamaktadır. Kardiyotokografi (CTG) verilerine dayanarak fetal sağlık durumunu tahmin eden model, Flask web framework'ü ile kullanıcı dostu bir web uygulamasına dönüştürülecektir.

### 🎯 Proje Hedefleri
- Fetal health risk faktörlerini analiz etmek
- Multi-class classification modeli geliştirmek
- Web tabanlı interaktif dashboard oluşturmak
- End-to-end deployment sağlamak

### 📊 Veri Seti Hakkında
- **Kaynak**: Fetal Health Classification Dataset (CTG)
- **Hedef**: Fetal health durumu tahmin (multi-class classification)
- **Sınıflar**: Normal, Suspect, Pathological
- **Özellikler**: Kardiyotokografi ölçümleri

---

## 🎯 PACE Aşama 1: PLAN (Planlama)

### 📋 İş Problemi Tanımlama
Fetal health monitoring gebelik döneminde kritik öneme sahiptir. Bu projede:
- **Ana Hedef**: CTG verilerine dayanarak fetal health durumunu sınıflandırmak
- **İş Değeri**: Erken risk tespit ve müdahale imkanı sağlamak
- **Başarı Metrikleri**: Model doğruluğu %85+ ve balanced performance

### 🔍 Veri Anlayışı ve Hipotezler
**Ana Hipotezler:**
1. Baseline fetal heart rate anormallikler risk göstergesi
2. Decelerations (yavaşlamalar) pathological durumları işaret eder
3. Heart rate variability fetal wellness göstergesi
4. Histogram özellikleri fetal distress belirteci olabilir

### 📈 Analitik Yaklaşım
- **Model Tipi**: Multi-class Classification (3 sınıf)
- **Değerlendirme Metrikleri**: Accuracy, Precision, Recall, F1-Score (weighted)
- **Deployment**: Flask web uygulaması ile real-time tahmin

### 🏥 Klinik Önem
- **Normal**: Sağlıklı fetal gelişim
- **Suspect**: İzlem gerektiren durum
- **Pathological**: Acil müdahale gerekli

---

In [ ]:
# 📚 Gerekli Kütüphanelerin İmport Edilmesi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn Kütüphaneleri
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, confusion_matrix, classification_report)

# Görselleştirme Ayarları
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

# Pandas Display Ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Tüm kütüphaneler başarıyla yüklendi!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 Numpy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")